In [ ]:
import pandas as pd
import numpy as np
import gc
import os
from tqdm import tqdm
from PIL import Image
from torchvision.utils import draw_bounding_boxes
from torchvision.utils import make_grid
from torchvision.io import read_image
from pathlib import Path
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
def rmse(y_pred,y_true):
    loss = np.sqrt(np.mean(np.square(y_true - y_pred), axis=0))
    return loss

#plt.rcParams["savefig.bbox"] = 'tight'
pd.set_option('mode.chained_assignment',None)
%matplotlib inline
import torch
data_dir='/kaggle/input/petfinder-pawpularity-score/'
dataset_path=data_dir
target='Pawpularity'

In [ ]:
if not os.path.exists('./models'):
    os.makedirs('./models')
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

# **Generating Pet Counts**

In [ ]:
!mkdir -p /root/.cache/torch/hub/ultralytics_yolov5_master
!cp -r ../input/yolo5x-pkl-file-for-pytorch/yolov5-master/yolov5-master/* /root/.cache/torch/hub/ultralytics_yolov5_master
!mkdir -p /root/.config/Ultralytics
!cp ../input/nfl-arial/Arial.ttf /root/.config/Ultralytics/Arial.ttf
!cp -r ../input/yolo-lung-detector/yolov5x.pt /root/.cache/torch/hub/ultralytics_yolov5_master/yolov5x.pt

In [ ]:
#model_yolo = torch.load('yolov5x.pkl')
#model_yolo = torch.load('../input/yolo5x-pkl-file-for-pytorch/yolov5x.pkl')
model_yolo =torch.hub.load('ultralytics/yolov5', 'yolov5x')
model_yolo.classes = [0,15,16] 
#torch.save(model_yolo,'yolov5x.pkl')
#model.conf = 0.25  # confidence threshold (0-1)
#model.iou = 0.45  # NMS IoU threshold (0-1)
 # filter by class, i.e. = [15, 16] for cats and dogs
    
#############To save the model for offline use ###########

gc.collect()

In [ ]:
def process_batch(df_source,img_source_dir,a_rang):
    gc.collect()
    torch.cuda.empty_cache()
    img_list =[Image.open(img_source_dir+df_source['Id'][j]+'.jpg') for j in a_rang]
    img_area=[float(img.size[0]*img.size[1]) for img in img_list] 
    results = model_yolo(img_list)
    results_pd=results.pandas().xyxy

    x_len=len(a_rang)
    for i in a_rang:
        results_tmp=results_pd[i%x_len]
        dog_count=results_tmp['name'][results_tmp['name']=='dog'].count()
        cat_count=results_tmp['name'][results_tmp['name']=='cat'].count()
        person_count=results_tmp['name'][results_tmp['name']=='person'].count()
        results_tmp['areas']=(results_tmp['xmax']-results_tmp['xmin'])*(results_tmp['ymax']-results_tmp['ymin'])/img_area[i%x_len]
        #print(results_tmp)
        max_area=results_tmp['areas'][(results_tmp['name']=='dog')|(results_tmp['name']=='cat')].max()
        min_area=results_tmp['areas'][(results_tmp['name']=='dog')|(results_tmp['name']=='cat')].min()
        average_area=results_tmp['areas'][(results_tmp['name']=='dog')|(results_tmp['name']=='cat')].mean()
        #print(max_area,min_area,average_area)
        #std_area=results_tmp['areas'][(results_tmp['name']=='dog')|(results_tmp['name']=='cat')].std()
        df_source.at[i,'nbrDogs']=dog_count
        df_source.at[i,'nbrCats']=cat_count
        df_source.at[i,'nbrPersons']=person_count
        df_source.at[i,'max_area']=max_area
        df_source.at[i,'min_area']=min_area
        df_source.at[i,'average_area']=average_area
        #df_source.at[i,'std_area']=std_area
        #print(df_source.iloc[i])

    return df_source

def generate_pet_counts(df_source,img_source_dir,bs):
    gc.collect()
    df_source['nbrCats']=0
    df_source['nbrDogs']=0
    df_source['nbrPersons']=0
    cat_columns=list(df_source.columns)
    df_source['max_area']=0.00001
    df_source['min_area']=0.00001
    df_source['average_area']=0.00001
    #df_source['std_area']=0
    cat_columns.remove('Id')
    cat_columns.append('nbrCats')
    cat_columns.append('nbrDogs')
    cat_columns.append('nbrPersons')
    df_source[cat_columns].astype('int8')
    df_source[['max_area','min_area','average_area']].astype('float')
        
    x_len=int(df_source.shape[0]/bs)
    for i in tqdm(range(x_len)):
        a_rang=range(i*bs,(i+1)*bs)
        df_source=process_batch(df_source,img_source_dir,a_rang) #multiples
    if ((df_source.shape[0]%bs)!=0):
        a_rang=range(x_len*bs,df_source.shape[0])
        df_source=process_batch(df_source,img_source_dir,a_rang)#the rest
    return df_source.fillna(0)
batch_size=200

In [ ]:
#trainSet = pd.read_csv(data_dir+'train.csv', low_memory=False)
#trainSet=trainSet.iloc[:100]
#gc.collect()
#trainSet=generate_pet_counts(trainSet,data_dir+'train/',batch_size)
#trainSet[['Id','nbrCats','nbrDogs']].head(5)
train_fname='../input/trainset-with-pet-count/train_with_areaFeatures.ftr'
trainSet=pd.read_feather(train_fname)
train_df=trainSet.copy()
train_df['norm_score'] = train_df['Pawpularity']/100
train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path+'train/'+x)+'.jpg')

In [ ]:
testSet = pd.read_csv(data_dir+'test.csv', low_memory=False)
testSet=generate_pet_counts(testSet,data_dir+'test/',batch_size)
testSet['path'] = testSet['Id'].map(lambda x:dataset_path+'test/'+str(x)+'.jpg')

# **Load Trained Model**

In [ ]:
import sys
import os
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
import gc
from fastai.vision.all import *
BATCH_SIZE = 16
seed=365
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def clean_mem():
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
def rmse(pred,target):
  return np.sqrt(np.mean(np.square(target - pred)))

def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(torch.sigmoid(input.flatten()), target))

def z_freeze(model):
  for name, param in model.named_parameters():
#    if name.startswith('head')  or name.startswith('classifier'): #or name.startswith('norm')
    #param.requires_grad = True  
#      continue
    param.requires_grad = False
def z_unfreeze(model):
  for name, param in model.named_parameters():
    param.requires_grad = True
def print_param_status(model):
  for i,x in enumerate(model.named_parameters()):
    name, param=x
    print(i,name, param.requires_grad)
    
def tuple_of_tensors_to_tensor(tuple_of_tensors):
    return  torch.stack(list(tuple_of_tensors), dim=0)
def calc_emb_size(x):
    return min(600, round(1.6 * x**0.56))

def get_training_score(learn):
  preds, _ = learn.get_preds(ds_idx=0)
  preds=np.squeeze(preds.float().numpy()*100)
  train_score=rmse(preds,learn.dls.train_ds.items['Pawpularity'])
  return train_score

In [ ]:
cat_list=['nbrCats', 'nbrDogs']#,'Eyes', 'Face']#,'Group',
max_cat=[15]*2#+[2]*2
#cat_list=['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory','Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'nbrCats', 'nbrDogs']
#max_cat=[2]*12+[15]*2
nb_cat=len(cat_list)

blocks=(ImageBlock(cls=PILImage),*(nb_cat*[CategoryBlock]),RegressionBlock)
getters_x=[ColReader('path', pref='', suff='')]
#,ColReader('Eyes'), ColReader('Face'),ColReader('Group'),ColReader('nbrCats'), ColReader('nbrDogs')]
for i in range(nb_cat): getters_x.append(ColReader(cat_list[i]))
getters_y=ColReader('norm_score')
dblock = DataBlock(blocks=blocks, get_x=getters_x,get_y=getters_y,  n_inp=nb_cat+1,# Set the number of inputs
              item_tfms=Resize(224),
                   #splitter=...,  #batch_tfms=...
)
dls = dblock.dataloaders(train_df, bs=BATCH_SIZE)
#dls.show_batch()
clean_mem()

In [ ]:
# From image_tabular module
class CNNTabularModel(Module):
    def __init__(self, cnn_model, tabular_model, layers, ps, out_sz):
        super().__init__()
        self.cnn_model = cnn_model
        self.tabular_model = tabular_model

        ps = ifnone(ps, [0]*len(layers))
        ps = [ps for i in range(len(layers))]
        sizes = layers + [out_sz]
        actns = [nn.ReLU(inplace=True) for _ in range(len(sizes)-2)] + [None]
        layers = []
        for n_in,n_out,dp,act in zip(sizes[:-1],sizes[1:], ps, actns):
            layers += LinBnDrop(n_in, n_out, bn=True, p=dp, act=act)
        self.layers = nn.Sequential(*layers)


    def forward(self, *x):
        x_image = self.cnn_model(x[0])
        x_tab = self.tabular_model(tuple_of_tensors_to_tensor(x[1:]).transpose(0,1)) 
        #print(type(x_image),type(x_tab))
        #<class 'fastai.torch_core.TensorImage'> <class 'fastai.torch_core.TensorCategory'>
        #print(torch.tensor(x_image),torch.tensor(x_tab))
        # Pytorch doesn't accept mixed Tensor types
        #.as_subclass(TensorBase)
        #torch.tensor(x_image),torch.tensor(x_tab) # gives a warning
        x_image, x_tab= x_image.as_subclass(TensorBase),x_tab.as_subclass(TensorBase)
        x = torch.cat((x_image, x_tab), 1)
        x = self.layers(x)
        return x
    
model_str=['resnet34','resnet50','efficientnet_b4','swin_large_patch4_window7_224']
cnn_out_sz_list=[512,2048,1792,1536]
####BATCH_SIZE = 128,128,128,16
model_num=3
image_model = create_model(model_str[model_num], pretrained=True, num_classes=0)
cnn_out_sz = cnn_out_sz_list[model_num]
clean_mem()
from fastai.tabular import *
from fastai.tabular.model import TabularModel  
# get embedding sizes of categorical data
tab_df=train_df[cat_list]

cont_names=[]
emb_szs = [(max_cat[i],calc_emb_size(max_cat[i])) for i in range(nb_cat)]
print('Embedding sizes: ',emb_szs)
# output size of the tabular model that will be concatenated with cnn model output
total_emb_size=sum([calc_emb_size(max_cat[i]) for i in range(nb_cat)])
tab_out_sz = 4#max(10,total_emb_size//2)

# use fastai functions to get a tabular model
tabular_model = TabularModel(emb_szs, len(cont_names), out_sz=tab_out_sz, layers=[6], ps=0.2) #max(total_emb_size,16)
#tabular_model
clean_mem()
# use gpu by default if available
integrated_model = CNNTabularModel(image_model,
                                  tabular_model,
                                  layers = [cnn_out_sz + tab_out_sz,512, 32],
                                  ps=0.4,
                                  out_sz=1).to(device)
clean_mem()
!cp ../input/trainset-with-pet-count/swin_01.pth models
learn = Learner(dls, integrated_model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse)#.to_fp16()
learn.load('swin_01',with_opt=False)
get_training_score(learn)

# **Inference and Submission**

In [ ]:
clean_mem()
test_dl=dls.test_dl(testSet)
preds, _ = learn.get_preds(dl=test_dl)
preds=np.squeeze(preds.float().numpy()*100)

In [ ]:
testSubmission=pd.DataFrame()
testSubmission['Id']=testSet['Id']
testSubmission[target]=preds
testSubmission[['Id', target]].to_csv('submission.csv',index = False)
testSubmission.head(5)